**Make plots of monitoring elements**

In particular:
- Limited to making plots of a single 1D ME.
- Make plots of single lumisections, a selected range of lumisections, all lumisections in a selected run, and a loop over selected runs.

In [ ]:
### imports

# external modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import importlib

# internal modules
sys.path.append('../utils')
import dataframe_utils as dfu
import plot_utils as pu
import hist_utils as hu
importlib.reload(dfu)
importlib.reload(pu)
importlib.reload(hu)

In [ ]:
### read the data
# OPTION 1: from previously stored file

# define input file
year = '2024'
dataset = 'ZeroBias-Run2024E-PromptReco-v1'
mename = 'PixelPhase1-Tracks-PXBarrel-charge_PXLayer_2'
shortmename = mename.replace('PixelPhase1-Tracks-PXBarrel-','')
filename = '{}-DQMIO-{}.parquet'.format(dataset, mename)
datadir = '../../SWAN_projects/ML4DQM-DC/data/dqmio2024_charge'
inputfile = os.path.join(datadir, filename)

# load the data
df = pd.read_parquet(inputfile)
df.head()

In [ ]:
### read the data
# OPTION 2: directly from DIALS

# imports and authentication
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters

creds = Credentials.from_creds_file()
dials = Dials(creds, workspace='tracker')

In [ ]:
### read the data
# OPTION2, continued

# make a filter
dataset = '/ZeroBias/Run2024E-PromptReco-v1/DQMIO'
mename = 'PixelPhase1/Tracks/PXBarrel/charge_PXLayer_2'
run_number = 381053
filters = LumisectionHistogram1DFilters(
    dataset=dataset,
    me=mename,
    run_number=run_number
)

data = dials.h1d.list_all(
    filters,
    enable_progress=True
)
df = data.to_pandas()
df.head()

In [ ]:
### print some dataframe properties

print('Number of runs: {}'.format(len(dfu.get_runs(df))))
print('Number of LS: {}'.format(len(dfu.get_ls(df))))
print('Avaialable MEs: {}'.format(dfu.get_menames(df)))

In [ ]:
### filter the data (optional)

# location of DCS-on json file
# note: make sure to modify or add the correct json files depending on your data
dcsjson = '/eos/user/c/cmsdqm/www/CAF/certification/Collisions24/DCSOnly_JSONS/dailyDCSOnlyJSON/Collisions24_13p6TeV_378981_381309_DCSOnly_TkPx.json'

# filter the data
# note: modify to your taste
dff = dfu.select_json(df, dcsjson)

print('Filtered number of runs: {}'.format(len(dfu.get_runs(dff))))
print('Filtered number of LS: {}'.format(len(dff)))
print('All available run numbers in selected data:\n{}'.format(dfu.get_runs(dff)))

In [ ]:
### settings for plot aesthetics

xaxtitle = 'Cluster charge (electrons)'
yaxtitle = 'Number of clusters'
caxtitle = 'Lumisection number'
extratext = 'BPIX L2'
extracmstext = 'Preliminary'
lumitext = '2024 (13.6 TeV)'
xmin = df['x_min'].values[0]
xmax = df['x_max'].values[0]

In [ ]:
### make plot of all lumisections in a selected run

# settings
run = 381053

# get the histograms
dfr = dfu.select_runs(dff,[run])
print('Selected number of LS: {}'.format(len(dfr)))
ls = np.array(dfr['ls_number'].values)
hists = np.vstack(dfr['data'].values)
print('Shape of hists array: {}'.format(hists.shape))
    
# plot raw histograms
fig,ax = pu.plot_hists_multi( hists.tolist(),
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitle, xaxtitlesize=17,
                                yaxtitle = yaxtitle, yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
pu.add_text( ax, 'Run {}'.format(run), (0.97,0.9), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, extratext, (0.97,0.82), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )
fig.tight_layout()
plt.show()
#fig.savefig('run{}.png'.format(run), dpi=300)
        
# plot processed histograms
hists = hu.normalizehists(hists)
fig,ax = pu.plot_hists_multi( hists.tolist(),
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitle, xaxtitlesize=17,
                                yaxtitle = yaxtitle+' (normalized)', yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
pu.add_text( ax, 'Run {}'.format(run), (0.97,0.9), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, extratext, (0.97,0.82), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )
fig.tight_layout()
plt.show()
#fig.savefig('run{}normalized.png'.format(run), dpi=300)

In [ ]:
### make plot of some selected lumisections (or a single lumisection) in a selected run

# settings
run = 381053
#ls = np.arange(60,80) # use either a single int or an iterable (e.g. list, range, array, etc.)
ls = 65 # use either a single int or an iterable (e.g. list, range, array, etc.)

# get the histogram(s)
dfr = dfu.select_runs(dff,[run])
if isinstance(ls, int): ls = [ls]
dfr = dfu.select_ls(dfr,ls)
print('Selected number of LS: {}'.format(len(dfr)))
ls = np.array(dfr['ls_number'].values)
hists = np.vstack(dfr['data'].values)
print('Shape of hists array: {}'.format(hists.shape))

# make the plot
fig,ax = pu.plot_hists_multi( hists.tolist(),
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitle, xaxtitlesize=17,
                                yaxtitle = yaxtitle, yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
pu.add_text( ax, 'Run {}, LS {}-{}'.format(run, ls[0], ls[-1]), (0.97,0.9), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, extratext, (0.97,0.82), fontsize=15, horizontalalignment='right' )
pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )
fig.tight_layout()
plt.show()
plt.show()

In [ ]:
### make plot of all lumisections in all runs in a loop

# get run numbers
runs = dfu.get_runs(dff)
print('Number of runs: '+str(len(runs)))

# do some selection
runs = runs[:1]
print('Will loop over following runs: '+str(runs))

for run in runs:
    
    # get the histograms
    dfr = dfu.select_runs(dff,[run])
    ls = np.array(dfr['ls_number'].values)
    hists = np.vstack(dfr['data'].values)
    
    # plot raw histograms
    fig,ax = pu.plot_hists_multi( hists.tolist(),
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitle, xaxtitlesize=17,
                                yaxtitle = yaxtitle, yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
    pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
    pu.add_text( ax, 'Run {}'.format(run), (0.97,0.9), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, extratext, (0.97,0.82), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )
    fig.tight_layout()
    plt.show()
    #fig.savefig('run{}.png'.format(run), dpi=300)
        
    # plot processed histograms
    hists = hu.normalizehists(hists)
    fig,ax = pu.plot_hists_multi( hists.tolist(),
                                colorlist=ls,
                                xlims=(xmin,xmax),
                                xaxtitle = xaxtitle, xaxtitlesize=17,
                                yaxtitle = yaxtitle+' (normalized)', yaxtitlesize=17,
                                caxtitle = caxtitle, caxtitlesize=15, caxtitleoffset=15,
                                ymaxfactor = 1,
                                legendsize = 14, ticksize = 14 )
    pu.add_cms_label( ax, pos=(0.05,1.01), extratext=extracmstext, fontsize=16 )
    pu.add_text( ax, 'Run {}'.format(run), (0.97,0.9), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, extratext, (0.97,0.82), fontsize=15, horizontalalignment='right' )
    pu.add_text( ax, lumitext, (0.71,1.01), fontsize=14 )
    fig.tight_layout()
    plt.show()
    #fig.savefig('run{}normalized.png'.format(run), dpi=300)